In [4]:
pip install pandas numpy requests requests-cache retry requests retry_requests openmeteo-requests

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 170 kB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 144 kB 59.4 MB/s eta 0:00:01
     |████████████████████████████████| 684 kB 31.8 MB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 18.3 MB/s eta 0:00:01
     |████████████████████████████████| 622 kB 34.8 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Open-Meteo Archive API
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 3.043092,
    "longitude": 101.441392,
    "start_date": "2020-01-01",
    "end_date": "2025-12-31",
    "hourly": "rain",
}

responses = openmeteo.weather_api(url, params=params)

# Process first location
response = responses[0]

# Extract hourly data
hourly = response.Hourly()
hourly_rain = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {
    "datetime": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left",
    ),
    "rain_mm": hourly_rain,
}

hourly_dataframe = pd.DataFrame(hourly_data)

# ✅ Save to CSV
output_file = "hourly_rainfall_2020_2025.csv"
hourly_dataframe.to_csv(output_file, index=False)

print(f"Hourly rainfall data saved to {output_file}")


Hourly rainfall data saved to hourly_rainfall_2020_2025.csv
